In [2]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [31]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [32]:
garbageCollectionDate

['AlpineShireCouncil-garbage-collection-zones.json',
 'Casey-garbage-collection-zones.geojson',
 'CentralGoldfieldsShireCouncil-garbage-collection-zones.json',
 'CityofGreatGeelong-garbage-collection-zones.json',
 'CityofGreaterBendigo-garbage-collection-zones.geojson',
 'FrankstonCityCouncil-garbage-collection-zones.json',
 'GreaterSheppartonCityCouncil-garbage-collection-zones.json',
 'HobsonsBayCityCouncil-garbage-collection-zones.json',
 'HorshamRuralShireCouncil-garbage-collection-zones.json',
 'LoddonShireCouncil-garbage-collection-zones.json',
 'Melbourne_Garbage_collection_zones.geojson',
 'NorthernGrampiansShireCouncil-garbage-collection-zones.json',
 'PortPhillip-garbage-collection-zones.geojson',
 'RuralCityofWangaratta-garbage-collection-zones.json',
 'Southern-Grampians-garbage-collection-zones.json',
 'StrathbogieShire-garbage-collection-zones.json',
 'WestWimmeraShireCouncil-garbage-collection-zones.json',
 'Whittlesea-garbage-collection-zones.geojson',
 'WodongaCouncil-

In [26]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

In [64]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [86]:
df = pd.read_json(garbageCollectionDate[0])

colNames = ['locality', 'long', 'lat', 'recyclable_colDay']
rows = [[]]*len(colNames)

alpineshireCD = pd.DataFrame(dict(zip(colNames, rows)))

df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])

df = df.explode('geo')
df['geo_nested'] = df.geo.apply(lambda x: isNested(x))

df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

In [87]:
df.drop(['geo', 'features', 'type', 'name', 'geo_nested'], axis = 1, inplace = True)

,properties,lat,long
0,"{'rec_day': 'Monday', 'rec_weeks': 2, 'rec_sta...",146.701728,-36.553711
0,"{'rec_day': 'Monday', 'rec_weeks': 2, 'rec_sta...",146.700669,-36.554071
0,"{'rec_day': 'Monday', 'rec_weeks': 2, 'rec_sta...",146.698555,-36.554790
0,"{'rec_day': 'Monday', 'rec_weeks': 2, 'rec_sta...",146.696335,-36.554010
0,"{'rec_day': 'Monday', 'rec_weeks': 2, 'rec_sta...",146.696153,-36.553826
...,...,...,...
37,"{'rub_day': 'Tuesday', 'rub_weeks': '1', 'rub_...",146.945418,-36.721081
37,"{'rub_day': 'Tuesday', 'rub_weeks': '1', 'rub_...",146.945283,-36.720832
37,"{'rub_day': 'Tuesday', 'rub_weeks': '1', 'rub_...",146.945149,-36.720583
37,"{'rub_day': 'Tuesday', 'rub_weeks': '1', 'rub_...",146.945014,-36.720335


In [20]:
df = pd.read_json('GarbageCollectionDate/'+i)
df['properties'] = df.features.apply(lambda x: x['properties'])
        
        try:

            df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])
        except: 
            df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0][0])
    
        collectionDate_geo = df[['properties', 'geo']].explode('geo')

        newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
        newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

        for key, val in dict(zip(newCols, newColsKeys)).items(): 
            try: 
                collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
            except: 
                collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])

        for colFreq in ['landfill_colFreq', 'recyclable_colFreq']: 
            collectionDate_geo[colFreq] = collectionDate_geo[colFreq].apply(lambda x: 'Every Second ' if x == '2' else 'Every ')

        collectionDate_geo['landfill_colDay'] = collectionDate_geo.landfill_colFreq + collectionDate_geo.landfill_colDay
        collectionDate_geo['recyclable_colDay'] = collectionDate_geo.recyclable_colFreq + collectionDate_geo.recyclable_colDay

        collectionDate_geo['lat'] = collectionDate_geo.geo.apply(lambda x: round(x[0], 6))
        collectionDate_geo['long'] = collectionDate_geo.geo.apply(lambda x: round(x[1], 6))

        collectionDate_geo = collectionDate_geo[['lat', 'long', 'landfill_colDay', 'recyclable_colDay']]

        tmp = lonlatSuburb.merge(collectionDate_geo)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
        .sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay'])\
        .sort_values(['locality'])

        type1CollectionDates = pd.concat([type1CollectionDates, tmp])
        
    except: 
        print(i)

WodongaCouncil-garbage-collection-zones.json
FrankstonCityCouncil-garbage-collection-zones.json
HobsonsBayCityCouncil-garbage-collection-zones.json
AlpineShireCouncil-garbage-collection-zones.json
Melbourne_Garbage_collection_zones.geojson
CentralGoldfieldsShireCouncil-garbage-collection-zones.json
Southern-Grampians-garbage-collection-zones.json
PortPhillip-garbage-collection-zones.geojson


In [21]:
firstSampleDf = type1CollectionDates.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(subset = ['locality', 'landfill_colDay', 'recyclable_colDay'])\
[['landfill_colDay', 'recyclable_colDay', 'postcode']]\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay'], 
                           ['landfill', 'recyclable'])))\
.melt(id_vars = 'postcode')

In [22]:
cols = ['bin_type', 'postcode', 'collection_time']

In [23]:
exportFolder = '../wrangledData/'

In [24]:
_ = firstSampleDf.drop_duplicates().reset_index(drop = True)\
.rename(columns = {'value':'collection_time', 
                  'variable': 'bin_type'})

In [187]:
firstSampleDf.drop_duplicates().reset_index(drop = True)\
.rename(columns = {'value':'collection_time', 
                  'variable': 'bin_type'})\
.to_csv(exportFolder + 'wasteCollectionDate_ver1.csv', index = False)



In [189]:
insertQuery = 'INSERT INTO wasteCollection(binInfo_bin_type, postcode, collection_time) VALUES\n'
values = ''

for i in _.index: 
    values = values + f'("{_.bin_type[i].strip()}", "{_.postcode[i].strip()}", "{_.collection_time[i].strip()}"),\n'.replace('"', "'")
    
    

In [191]:
print(insertQuery + values)

INSERT INTO wasteCollection(binInfo_bin_type, postcode, collection_time) VALUES
('landfill', '3371', 'Every Second Thursday'),
('landfill', '3143', 'Every Tuesday'),
('landfill', '3183', 'Every Tuesday'),
('landfill', '3463', 'Every Second Wednesday'),
('landfill', '3691', 'Every Second Wednesday'),
('landfill', '3465', 'Every Second Wednesday'),
('landfill', '3186', 'Every Wednesday'),
('landfill', '3472', 'Every Second Monday'),
('landfill', '3464', 'Every Second Wednesday'),
('landfill', '3691', 'Every Second Monday'),
('landfill', '3161', 'Every Tuesday'),
('landfill', '3465', 'Every Second Tuesday'),
('landfill', '3185', 'Every Wednesday'),
('landfill', '3184', 'Every Wednesday'),
('landfill', '3695', 'Every Second Wednesday'),
('landfill', '3688', 'Every Second Friday'),
('landfill', '3688', 'Every Second Monday'),
('landfill', '3688', 'Every Second Tuesday'),
('landfill', '3364', 'Every Second Wednesday'),
('landfill', '3691', 'Every Second Tuesday'),
('landfill', '8001', 'Every

In [135]:
df = pd.read_json('GarbageCollectionDate/FrankstonCityCouncil-garbage-collection-zones.json')

In [136]:
df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0][0])
collectionDate_geo = df[['properties', 'geo']].explode('geo')

newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    try: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
    except: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])


In [139]:
df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])
collectionDate_geo = df[['properties', 'geo']].explode('geo')

newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    try: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
    except: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])
    
for colFreq in ['landfill_colFreq', 'recyclable_colFreq']: 
    collectionDate_geo[colFreq] = collectionDate_geo[colFreq].apply(lambda x: 'Every Second ' if x == '2' else 'Every ')
    

In [143]:
collectionDate_geo

,properties,geo,landfill_colDay,landfill_colFreq,recyclable_colDay,recyclable_colFreq
0,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[[145.205084864, -38.1446568331], [145.2283478...",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.1441279116, -38.1474438406]",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.1447778394, -38.1477778302]",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.144971836, -38.1479238793]",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.1450769143, -38.1480438368]",Every Tuesday,Every,Every Tuesday,Every
...,...,...,...,...,...,...
5,"{'rub_day': 'Thursday', 'rub_weeks': 1, 'rub_s...","[145.118492867, -38.1456328741]",Every Thursday,Every,Every Thursday,Every
6,"{'rub_day': 'Thursday', 'rub_weeks': 1, 'rub_s...","[[145.1328198908, -38.1534268326], [145.133164...",Every Thursday,Every,Every Thursday,Every
7,"{'rub_day': 'Monday', 'rub_weeks': 1, 'rub_sta...","[[145.1572299083, -38.1126808407], [145.160402...",Every Monday,Every,Every Monday,Every
8,"{'rub_day': 'Friday', 'rub_weeks': 1, 'rub_sta...","[[145.1968809036, -38.1024838245], [145.172497...",Every Friday,Every,Every Friday,Every


In [142]:
collectionDate_geo['landfill_colDay'] = collectionDate_geo.landfill_colFreq + collectionDate_geo.landfill_colDay
collectionDate_geo['recyclable_colDay'] = collectionDate_geo.recyclable_colFreq + collectionDate_geo.recyclable_colDay

collectionDate_geo['lat'] = collectionDate_geo.geo.apply(lambda x: round(x[0], 6))
collectionDate_geo['long'] = collectionDate_geo.geo.apply(lambda x: round(x[1], 6))

collectionDate_geo = collectionDate_geo[['lat', 'long', 'landfill_colDay', 'recyclable_colDay']]

tmp = lonlatSuburb.merge(collectionDate_geo)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay'])\
.sort_values(['locality'])

TypeError: type list doesn't define __round__ method

In [78]:
df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])
collectionDate_geo = df[['properties', 'geo']].explode('geo')

newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    try: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
    except: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])
    
for colFreq in ['landfill_colFreq', 'recyclable_colFreq']: 
    collectionDate_geo[colFreq] = collectionDate_geo[colFreq].apply(lambda x: 'Every Second ' if x == '2' else 'Every ')
    
collectionDate_geo['landfill_colDay'] = collectionDate_geo.landfill_colFreq + collectionDate_geo.landfill_colDay
collectionDate_geo['recyclable_colDay'] = collectionDate_geo.recyclable_colFreq + collectionDate_geo.recyclable_colDay

collectionDate_geo['lat'] = collectionDate_geo.geo.apply(lambda x: round(x[0], 6))
collectionDate_geo['long'] = collectionDate_geo.geo.apply(lambda x: round(x[1], 6))

collectionDate_geo = collectionDate_geo[['lat', 'long', 'landfill_colDay', 'recyclable_colDay']]

tmp = lonlatSuburb.merge(collectionDate_geo)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay'])\
.sort_values(['locality'])

type1CollectionDates = pd.concat([type1CollectionDates, tmp])